# RAG - ColBERT IR, gpt tubo 3.5

## IR

### 라이브러리 설치 및 임포트

In [1]:
# 필요한 라이브러리 import

from colbert import Indexer, Searcher
from colbert.infra import Run, RunConfig, ColBERTConfig
from colbert.data import Queries, Collection
import colbert
import json

In [2]:
with open("/data/ephemeral/home/upstage-ai-advanced-ir2/data/documents.jsonl") as f:
    docs = [json.loads(line) for line in f]

In [3]:
# docs 에서 줄바꿈 문자를 제거 (학습 할때 에러 방지)
for doc in docs:
    doc['content'] = doc['content'].replace("\n", "")
    doc['content'] = doc['content'].replace("\r", "")
print(docs[0])
collection = [doc['content'] for doc in docs]
print(collection[0])
docid_list = [doc['docid'] for doc in docs]
print(docid_list[0])
src_list = [doc['src'] for doc in docs]
src_list[0]    

{'docid': '42508ee0-c543-4338-878e-d98c6babee66', 'src': 'ko_mmlu__nutrition__test', 'content': '건강한 사람이 에너지 균형을 평형 상태로 유지하는 것은 중요합니다. 에너지 균형은 에너지 섭취와 에너지 소비의 수학적 동등성을 의미합니다. 일반적으로 건강한 사람은 1-2주의 기간 동안 에너지 균형을 달성합니다. 이 기간 동안에는 올바른 식단과 적절한 운동을 통해 에너지 섭취와 에너지 소비를 조절해야 합니다. 식단은 영양가 있는 식품을 포함하고, 적절한 칼로리를 섭취해야 합니다. 또한, 운동은 에너지 소비를 촉진시키고 근육을 강화시킵니다. 이렇게 에너지 균형을 유지하면 건강을 유지하고 비만이나 영양 실조와 같은 문제를 예방할 수 있습니다. 따라서 건강한 사람은 에너지 균형을 평형 상태로 유지하는 것이 중요하며, 이를 위해 1-2주의 기간 동안 식단과 운동을 조절해야 합니다.'}
건강한 사람이 에너지 균형을 평형 상태로 유지하는 것은 중요합니다. 에너지 균형은 에너지 섭취와 에너지 소비의 수학적 동등성을 의미합니다. 일반적으로 건강한 사람은 1-2주의 기간 동안 에너지 균형을 달성합니다. 이 기간 동안에는 올바른 식단과 적절한 운동을 통해 에너지 섭취와 에너지 소비를 조절해야 합니다. 식단은 영양가 있는 식품을 포함하고, 적절한 칼로리를 섭취해야 합니다. 또한, 운동은 에너지 소비를 촉진시키고 근육을 강화시킵니다. 이렇게 에너지 균형을 유지하면 건강을 유지하고 비만이나 영양 실조와 같은 문제를 예방할 수 있습니다. 따라서 건강한 사람은 에너지 균형을 평형 상태로 유지하는 것이 중요하며, 이를 위해 1-2주의 기간 동안 식단과 운동을 조절해야 합니다.
42508ee0-c543-4338-878e-d98c6babee66


'ko_mmlu__nutrition__test'

### ColBER 색인

In [4]:
import torch
print(torch.cuda.is_available())  # True여야 합니다.
print(torch.cuda.get_device_name(0))  # 'Tesla T4'와 같은 GPU 이름이 나와야 합니다.


True
NVIDIA GeForce RTX 3090


In [5]:
import torch
print(torch.cuda.is_available())  # True가 출력되면 GPU 사용이 가능함을 의미합니다.
print(torch.cuda.device_count())  # 사용 가능한 GPU의 수를 출력합니다.


True
1


In [6]:
# 위에서 확인한 학습된 모델의 위치를 checkpoint에 넣어줌
checkpoint = 'experiments/sample_ko_new_2/none/2024-10/16/07.20.10/checkpoints/colbert'
experiment = 'sample_ko_new_2'
index_name = 'science_common_sense_2'
nbits = 2

In [7]:


with Run().context(RunConfig(nranks=1, experiment=experiment)):  # nranks specifies the number of GPUs to use
    config = ColBERTConfig(nbits=nbits, kmeans_niters=4) # kmeans_niters specifies the number of iterations of k-means clustering; 4 is a good and fast default.
                                                                                # Consider larger numbers for small datasets.
    indexer = Indexer(checkpoint = checkpoint, config = config)
    indexer.index(name = index_name, collection = collection, overwrite=True)



[Oct 16, 15:15:53] #> Note: Output directory /data/ephemeral/home/upstage-ai-advanced-ir2/experiments/sample_ko_new/indexes/science_common_sense already exists


[Oct 16, 15:15:53] #> Will delete 10 files already at /data/ephemeral/home/upstage-ai-advanced-ir2/experiments/sample_ko_new/indexes/science_common_sense in 20 seconds...


In [11]:
# 검색을 수행하기 위한 Searcher 객체를 생성합니다.
# ColBERT 인덱스를 기반으로 검색 작업을 수행합니다.

with Run().context(RunConfig(experiment=experiment)):  # 실행 환경을 설정합니다. 여기서는 'notebook'이라는 이름으로 실험을 정의합니다.
    searcher = Searcher(index=index_name, collection=collection)  # 지정된 인덱스와 컬렉션을 사용하여 Searcher 객체를 초기화합니다.


[Oct 17, 01:32:55] #> Loading codec...
[Oct 17, 01:32:55] Loading decompress_residuals_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...
[Oct 17, 01:32:55] Loading packbits_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...
[Oct 17, 01:32:56] #> Loading IVF...
[Oct 17, 01:32:56] #> Loading doclens...


100%|██████████| 1/1 [00:00<00:00, 1036.65it/s]

[Oct 17, 01:32:56] #> Loading codes and residuals...



100%|██████████| 1/1 [00:00<00:00, 37.47it/s]


### Colbert 색인 문서 검색

In [8]:
# # 문서 검색 test
# queries = ["나무 분류하는 방법은?"]  # 검색할 질의 목록입니다.

# # 각 질의에 대해 검색을 수행합니다.
# for query in queries:
#     print(f"#> {query}")  # 현재 검색 질의를 출력합니다.

#     # 검색 질의에 대한 상위 5개의 문서를 검색합니다.
#     results = searcher.search(query, k=3)  # k는 검색 결과에서 반환할 상위 문서의 개수를 의미합니다.

#     # 검색된 상위 k개의 문서를 출력합니다.
#     for passage_id, passage_rank, passage_score in zip(*results):
#         # 검색 결과의 순위, 점수 및 해당 문서를 출력합니다.
#         print(f"{docid_list[passage_id]}\t [{passage_rank}] \t\t {passage_score:.1f} \t\t {searcher.collection[passage_id]}")


In [12]:
from openai import OpenAI
import traceback
from dotenv import load_dotenv
import os
import json

load_dotenv(dotenv_path='/data/ephemeral/home/upstage-ai-advanced-ir2/OPENAI_API_KEY.env')

openai_api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI()  
llm_model = "gpt-3.5-turbo-1106" #"gpt-4-turbo-2024-04-09" #"gpt-3.5-turbo-1106" #"gpt-4-turbo-2024-04-09"

persona_qa = """
## Role: 지식 백과사전

## Instructions
- 사용자의 이전 메시지 정보 및 주어진 Reference 정보를 활용하여 간결하게 답변을 생성한다.
- 주어진 검색 결과 정보로 대답할 수 없는 경우는 정보가 부족해서 답을 할 수 없다고 대답한다.
- 사용자가 사용한 언어로 답변을 생성한다.
"""


# 질의 쿼리는 여기에 추가(종결어미)
persona_function_calling = """
## Role: 과학 상식 전문가

## Instruction
- 사용자가 대화를 통해 지식에 관한 주제로 질문하면 search api를 호출할 수 있어야 한다.
- 지식과 관련되지 않은 나머지 대화 메시지에는 적절한 대답을 생성한다.
"""

tools = [
    {
        "type": "function",  
        "function": {
            "name": "search",  
            "description": "search relevant documents",  
            "parameters": {
                "properties": {  
                    "standalone_query": {
                        "type": "string",  
                        "description": "사용자 메세지 기록으로부터 검색에 사용할 적합한 최종 질의 생성"
                    }
                },
                "required": ["standalone_query"],  
                "type": "object" 
            }
        }
    },
]

def crossencoder_rerank(query, topk_results):
    cross_encoder_inputs = [(query, searcher.collection[passage_id]) for passage_id, _, _ in zip(*topk_results)]
    
    # Cross Encoder로 재랭킹
    rerank_scores = cross_encoder_model.predict(cross_encoder_inputs)
    
    # 점수를 float으로 변환
    rerank_scores = [float(score) for score in rerank_scores]

    # ColBERT 결과에서 docid와 점수를 함께 반환
    reranked_topk = sorted(zip(topk_results[0], rerank_scores), key=lambda x: x[1], reverse=True)
    
    # 상위 3개의 문서의 docid만 추출
    return [docid_list[passage_id] for passage_id, _ in reranked_topk[:3]]


def answer_question(messages):
    print("\n=== Answer Question ===")
    print("Messages: ", messages)

    response = {"standalone_query": "", "topk": [], "references": [], "answer": ""}

    msg = [{"role": "system", "content": persona_function_calling}] + messages
    try:
        result = client.chat.completions.create(
            model=llm_model,
            messages=msg,
            tools=tools,
            temperature=0,
            seed=1,
            timeout=10
        )
        print("LLM Response: ", result)
    except Exception as e:
        traceback.print_exc()
        return response

    response_message = result.choices[0].message

    if response_message.tool_calls:
        for tool_call in response_message.tool_calls:
            function_args = json.loads(tool_call.function.arguments)
            standalone_query = function_args.get("standalone_query")
            #print("Standalone Query: ", standalone_query)

            # ColBERT 검색 실행
            results = searcher.search(standalone_query, k=20)
            #print("ColBERT Search Results: ", results)
            #print("searcher : ", searcher)
            response["standalone_query"] = standalone_query
            retrieved_context = []

            # ColBERT 검색 결과에서 문서 ID와 점수 저장
            for passage_id, passage_rank, passage_score in zip(*results):
                retrieved_context.append(searcher.collection[passage_id])
                response["references"].append({"score": float(passage_score), "content": searcher.collection[passage_id]})
            #print("Retrieved Context: ", retrieved_context)

            # Cross-Encoder로 재랭킹
            topk = crossencoder_rerank(standalone_query, results)
            response["topk"] = topk

            # 도구 호출 결과로 LLM이 답변을 생성하도록 다시 요청
            final_prompt = [{"role": "user", "content": f"이 문서를 기반으로 요약한 답변을 생성해줘: {retrieved_context}"}]
            try:
                final_response = client.chat.completions.create(
                    model=llm_model,
                    messages=final_prompt,
                    temperature=0,
                    seed=1,
                    timeout=20
                )
                response["answer"] = final_response.choices[0].message.content
            except Exception as e:
                traceback.print_exc()
    else:
        response["answer"] = response_message.content if response_message.content else ""

    #print("Response: ", response)
    return response



# eval_rag 함수: question 데이터를 파일에서 읽어와 처리한 후 결과를 output에 저장
def eval_rag(eval_filename, output_filename):
    with open(eval_filename, 'r', encoding='utf-8') as f, open(output_filename, 'w', encoding='utf-8') as of:
        idx = 0
        for line in f:
            j = json.loads(line)
            print(f'Test {idx}\nQuestion: {j["msg"]}')

            # 메시지를 처리하여 검색 결과 및 답변 생성
            messages = [{"role": "user", "content": msg["content"]} for msg in j["msg"]]
            response = answer_question(messages)

            # 결과를 출력 파일에 저장
            output = {
                "eval_id": j["eval_id"],
                "standalone_query": response["standalone_query"],
                "topk": response["topk"],
                "answer": response["answer"],
                "references": response["references"][:3]
            }
            of.write(f'{json.dumps(output, ensure_ascii=False)}\n')
            idx += 1

# Cross-encoder model loading
from sentence_transformers import CrossEncoder
cross_encoder_model = CrossEncoder('output/klue-cross-encoder-v1_13q', max_length=512)



In [ ]:
# 최종 실행 예시
eval_rag('/data/ephemeral/home/upstage-ai-advanced-ir2/data/eval_copy.jsonl',
         '/data/ephemeral/home/upstage-ai-advanced-ir2/submission_test/prompt_test.csv')